In [ ]:
# Chap 4
(exception handling)

## Goals
learn the basic principles for *raising and handling errors functionally*.

The big idea is
- we can represent failures and excetipons with *ordinary values*, and
- we can write *higher-order functions* that abstract out common patterns of error handling and recovery,
- while preserving the benefit of consolidation of error-handling logic.

Recreate *Option* and *Either* to enhance your understading of how these types can be ...

## Non-RT Expression
---
```scala
def failingFn(i: Int): Int = {
  val y: Int = throw new Exception("fail!")
  try {
    val x = 42 + 5
    x+y
  }
  catch { case e: Exception => 43 }
}
```
---


The Good and The Bad of Exceptions
- Exceptions break RT and introduce context dependence, requiring non-local ...

## Option Type
Represent explicityly in the return type that a function may not always have an answer.
- deferring to the caller for the error-handling strategy.

---
```scala
sealed trait Option[+A]

case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]
```
---

In [2]:
def mean(xs: Seq[Double]): Option[Double] =
  if (xs.isEmpty) None
  else Some(xs.sum / xs.length)

defined function mean

## Partial function vs Total function

### Partial function
```scala
def mean(xs: Seq[Double]): Double =
  if (xs.isEmpty) throw ArithmeticException
  else xs.sum / xs.length
```
---
### Total function
```scala
def mean(xs: Seq[Double]): Option[Double] =
  if (xs.isEmpty) None
  else Some(xs.sum / xs.length)
```

## 참고 reftree

In [11]:
sealed trait Option[+A]{
  def map[B](f: A => B): Option[B] = this match {
    case Some(v) => Some(f(v))
    case None => None
  }
  
//  def getOrElse(default: A): A = ???  // default 파라미터에 타입 A를 쓸 수 없음 에러남, 
                                      // contravariant자리에 covariant가 나와서
  def getOrElse[B >: A](default: => B): B = this match {
    case Some(v) => v
    case None => default
  }
  
  def flatMap[B](f: A => Option[B]): Option[B] = this match {
    case Some(v) => f(v)
    case None => None
  }
  
  def flatMap_[B](f: A => Option[B]): Option[B] = 
    map(f).getOrElse(None)
    
  def orElse[B >: A](default: =>Option[B]): Option[B] = 
    map(Option(_)) getOrElse default
  
  def filter(f: A => Boolean): Option[A] =
    flatMap(a => if(f(a)) this else None) 
}

case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]

object Option {
  def none[A]: Option[A] = None
  def some[A](a: A): Option[A] = Some(a)
  def apply[A](a: A): Option[A] = Some(a)
  def map[A, B](fa: Option[A])(f: A => B): Option[B] = 
    fa map f
  def lift[A, B](f: A => B): Option[A] => Option[B] =
    map(_)(f)
//     (a: Option[A]) =>map(a)(f)
}

defined trait Option
defined class Some
defined object None
defined object Option

In [4]:
Some(10).map(_ + 20)

res3: Option[Int] = Some(30)

## Lifting
``` scala
def lift[A,B](f: A => B): Option[A] => Option[B] =???
```